<a href="https://colab.research.google.com/github/DmtrRm/IAAC2024_tutorials/blob/main/kaggle_complete2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIBRARIES IMPORT**

In [10]:
import calendar
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

**FUNCTIONS THAT WILL BE USED**

In [11]:
def encode_binary_categorical(data_set):
  binary_Lbls=[]
  categor_Lbls=[]

  for c in data_set.columns:

    if len(data_set[c].unique())<=3 and any(x=="Yes" for x in data_set[c].unique()):
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=1-res
      binary_Lbls.append(c)

    elif data_set[c].dtype=="object" and c != "day":
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=res
      categor_Lbls.append(c)

  return categor_Lbls

In [12]:
def find_pair_list(corralation,cor_value):


  pair_list=[]
  x=corralation.shape[0]
  y=corralation.shape[1]

  for i in range(0,x):
    for j in range(0,y):
      if (corralation.iloc[i,j]>=cor_value or corralation.iloc[i,j]<=(-cor_value)) and corralation.iloc[i,j]!=1 :
        pair_list=pair_list+[(sorted([corralation.columns[i], corralation.columns[j]]))]

  groups = []

  for pair in pair_list:
    added_to_group = False
    for group in groups:
      if any(variable in group for variable in pair):
        group.extend(pair)
        group = list(set(group))
        added_to_group = True
        break
    if not added_to_group:
      groups.append(pair)

  result = [list(set(group)) for group in groups]
  return result

In [17]:
def data_cleaning(i):


## dropping same value clm
  for c in i.columns:
   if len(i[c].unique())==1:
    i=i.drop([c], axis=1)

## encoding
  cLbls_i=encode_binary_categorical(i)

## droping dublicates
  result_i=find_pair_list(i.corr(),0.9)
  print(result_i)
  columns_to_drop = sum([x[0:-1] for x in result_i],[])
  i=i.drop(columns=columns_to_drop)

## filling the NaN
  cols_to_drop = []
  for c in i.columns:
    if len(i[c].isna().unique())>1:
      cols_to_drop.append(c)
  print(cols_to_drop)

  if not cols_to_drop==False:
    for n in cols_to_drop:
      X = i.loc[i[n].isna()==False].drop(columns=cols_to_drop)
      Y = i.loc[i[n].isna()==False][n]
      missing_X = i.loc[i[n].isna()].drop(columns=cols_to_drop)
      tree = DecisionTreeRegressor()
      tree.fit(X, Y)

      zipped = [(x, i) for i,x in sorted(zip(tree.feature_importances_, X), reverse=True)]

      i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)

  return i

**DATA UPLOADING**

In [42]:
metadata = pd.read_csv("/content/DATA/metadata.csv", index_col=0)
meters = pd.read_csv("/content/DATA/meters.csv", index_col=0)
test = pd.read_csv("/content/DATA/test.csv", index_col=0)
train = pd.read_csv("/content/DATA/train.csv", index_col=0)
weather = pd.read_csv("/content/DATA/weather.csv", index_col=0)

In [43]:
weather[['day', 'time']] = weather['timestamp'].str.split(' ', expand=True)

In [44]:
##dropping day
wth_day = weather["day"]
weather=weather.drop(columns="day")

train_day = train["day"]
train=train.drop(columns="day")

**DATA CLEANING**

In [45]:
metadata = data_cleaning(metadata)
train = data_cleaning(train)
weather = data_cleaning(weather)

[['sqft', 'sqm'], ['timezone', 'lng', 'heatingtype', 'lat']]
['site_id_kaggle', 'yearbuilt', 'numberoffloors', 'occupants']
[]
[]


<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-inpu

[]
['airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']


<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-17-7fc57c5a7bb4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-inpu

In [46]:
## inserting day
weather.insert(loc=1, column='day', value=wth_day)
train.insert(loc=1, column='day', value=train_day)



In [47]:
weather.head()

,timestamp,day,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,time
18121,0,2016-02-01,3,11.3,0.9,10.5,0.0,1024.0,160.5,3.4,1
18122,1,2016-02-01,3,12.3,0.6,10.8,0.0,1023.8,170.8,5.2,2
18123,2,2016-02-01,3,12.1,0.6,9.7,0.0,1022.7,180.2,5.1,3
18124,3,2016-02-01,3,12.4,0.7,9.8,0.0,1021.8,160.1,5.4,4
18125,4,2016-02-01,3,11.6,0.5,9.3,0.0,1021.6,170.1,5.1,5


In [49]:
weather["day"] = weather["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
weather["year"] = weather["day"].apply(lambda s: s.year)
weather["month"] = weather["day"].apply(lambda s: s.month)
weather["day_number"] = weather["day"].apply(lambda s: s.day)
weather["weekday"] = weather["day"].apply(lambda s: s.weekday() in [5,6])

In [50]:
site_ids = weather["site_id"].unique()
dfs = []

for site_id in site_ids:
    w_stats = weather.loc[weather["site_id"]==site_id].set_index("day").resample('D')[['airTemperature',
                                                                        "windSpeed"]].agg(['min', 'max', "mean"]).to_records()
    w_stats_df = pd.DataFrame(w_stats)
    w_stats_df["site_id"] = site_id
    dfs.append(w_stats_df)

w_df = pd.concat(dfs, axis=0)  # concatenating the resulting dataframes to have one

print("Shape:", w_df.shape)
w_df.head()

Shape: (3495, 8)


,day,"('airTemperature', 'min')","('airTemperature', 'max')","('airTemperature', 'mean')","('windSpeed', 'min')","('windSpeed', 'max')","('windSpeed', 'mean')",site_id
0,2016-02-01,9.1,14.2,11.782609,2.9,10.4,6.343478,3
1,2016-02-02,6.0,12.0,9.700000,2.7,13.0,8.266667,3
2,2016-02-03,8.1,13.9,12.004167,5.8,13.1,9.887500,3
3,2016-02-04,2.4,10.4,6.925000,3.1,10.7,6.433333,3
4,2016-02-05,9.8,12.7,11.591667,9.0,13.5,10.316667,3


In [9]:
weather["day"] = weather["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
weather["year"] = weather["day"].apply(lambda s: s.year)
weather["month"] = weather["day"].apply(lambda s: s.month)
weather["day_number"] = weather["day"].apply(lambda s: s.day)
weather["weekday"] = weather["day"].apply(lambda s: s.weekday() in [5,6])

In [10]:
train["day"] = train["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
train["year"] = train["day"].apply(lambda s: s.year)
train["month"] = train["day"].apply(lambda s: s.month)
train["day_number"] = train["day"].apply(lambda s: s.day)
train["weekday"] = train["day"].apply(lambda s: s.weekday() in [5,6])

**DATA MERGING**